# Control de rutas Greedy (baseline y k rutas)
Este cuaderno orquesta la ejecución de:
- Baseline greedy (1 ruta) con `baseline_greedy.py`.
- Greedy multi‑ruta k con `src/greedy_k_routes.py`.

Los resultados se guardan en `experiments/tables/` y se muestran tablas de resumen.

In [5]:
from pathlib import Path
import sys, subprocess
import pandas as pd

# Parámetros de entrada
instances = ['hptoptw-j11a.csv','hptoptw-j11b.csv','hptoptw-j16a.csv','hptoptw-j21a.csv']
baseline_runs = 3
use_k_from_csv = True  # si False, usa k_override
k_override = 2
return_to_depot = True

# Detectar raíz del repo de forma robusta (buscando baseline_greedy.py)
repo_root = Path.cwd()
candidates = [repo_root, repo_root.parent, repo_root.parent.parent, repo_root.parent.parent.parent]
for base in candidates:
    if (base / 'baseline_greedy.py').exists():
        repo_root = base
        break
data_syn = repo_root / 'data' / 'synthetic'
tables_dir = repo_root / 'experiments' / 'tables'
tables_dir.mkdir(parents=True, exist_ok=True)

baseline_script = repo_root / 'baseline_greedy.py'
k_routes_script = repo_root / 'src' / 'greedy_k_routes.py'
baseline_script.as_posix(), k_routes_script.as_posix()


('c:/Users/juann/OneDrive - Pontificia Universidad Javeriana/Trabajo de Grado/ttdp-trabajo-grado/baseline_greedy.py',
 'c:/Users/juann/OneDrive - Pontificia Universidad Javeriana/Trabajo de Grado/ttdp-trabajo-grado/src/greedy_k_routes.py')

## Ejecutar baseline (1 ruta)

In [6]:
rows = []
for fname in instances:
    name = Path(fname).stem
    fpath = data_syn / fname
    if not fpath.exists():
        print('[WARN] No existe', fpath.as_posix()); continue
    cmd = [sys.executable, str(baseline_script), '--file', str(fpath), '--name', name, '--runs', str(baseline_runs)]
    print('>>', ' '.join(cmd))
    res = subprocess.run(cmd, capture_output=True, text=True, cwd=str(repo_root))
    if res.returncode != 0:
        print('[ERROR]', res.stderr.strip())
    else:
        print(res.stdout.strip())
# Mostrar tablas resultantes
outs = sorted(tables_dir.glob('baseline_greedy_*.csv'))
print('Encontrados baseline CSV:', len(outs))
pd.concat([pd.read_csv(p) for p in outs if p.exists()], ignore_index=True).head() if outs else None


>> c:\Users\juann\anaconda3\python.exe c:\Users\juann\OneDrive - Pontificia Universidad Javeriana\Trabajo de Grado\ttdp-trabajo-grado\baseline_greedy.py --file c:\Users\juann\OneDrive - Pontificia Universidad Javeriana\Trabajo de Grado\ttdp-trabajo-grado\data\synthetic\hptoptw-j11a.csv --name hptoptw-j11a --runs 3
[hptoptw-j11a] run=0 feasible=True profit=100.00 time=607.54 stops=5 compute=0.0079s
[hptoptw-j11a] run=1 feasible=True profit=100.00 time=607.54 stops=5 compute=0.0049s
[hptoptw-j11a] run=2 feasible=True profit=100.00 time=607.54 stops=5 compute=0.0038s
Saved -> experiments\tables\baseline_greedy_hptoptw-j11a.csv
>> c:\Users\juann\anaconda3\python.exe c:\Users\juann\OneDrive - Pontificia Universidad Javeriana\Trabajo de Grado\ttdp-trabajo-grado\baseline_greedy.py --file c:\Users\juann\OneDrive - Pontificia Universidad Javeriana\Trabajo de Grado\ttdp-trabajo-grado\data\synthetic\hptoptw-j11b.csv --name hptoptw-j11b --runs 3
[hptoptw-j11b] run=0 feasible=True profit=100.00 tim

,instance,run,feasible,profit,route_time,stops,compute_sec
0,hptoptw-j11a,0,True,100.0,607.540659,5,0.007933
1,hptoptw-j11a,1,True,100.0,607.540659,5,0.004939
2,hptoptw-j11a,2,True,100.0,607.540659,5,0.003810
3,hptoptw-j11b,0,True,100.0,479.079872,4,0.003231
4,hptoptw-j11b,1,True,100.0,479.079872,4,0.004177


## Ejecutar greedy multi‑ruta k

In [7]:
outs_k = []
for fname in instances:
    name = Path(fname).stem
    fpath = data_syn / fname
    assert fpath.exists(), f'No existe {fpath}'
    cmd = [sys.executable, str(k_routes_script), '--file', str(fpath), '--name', name]
    if not use_k_from_csv:
        cmd += ['--k', str(k_override)]
    if return_to_depot:
        cmd += ['--return-to-depot']
    print('>>', ' '.join(cmd))
    res = subprocess.run(cmd, capture_output=True, text=True, cwd=str(repo_root))
    if res.returncode != 0:
        print('[ERROR]', res.stderr.strip())
    else:
        print(res.stdout.strip())
outs_k = sorted(tables_dir.glob('greedy_k_*.csv'))
print('Encontrados k-rutas CSV:', len(outs_k))
pd.concat([pd.read_csv(p) for p in outs_k if p.exists()], ignore_index=True).head() if outs_k else None


>> c:\Users\juann\anaconda3\python.exe c:\Users\juann\OneDrive - Pontificia Universidad Javeriana\Trabajo de Grado\ttdp-trabajo-grado\src\greedy_k_routes.py --file c:\Users\juann\OneDrive - Pontificia Universidad Javeriana\Trabajo de Grado\ttdp-trabajo-grado\data\synthetic\hptoptw-j11a.csv --name hptoptw-j11a --return-to-depot
[hptoptw-j11a] route=0 feasible=True profit=100.00 time=607.54 stops=5 compute=0.0051s
[hptoptw-j11a] route=1 feasible=True profit=40.00 time=245.39 stops=2 compute=0.0023s
[hptoptw-j11a] k=2 TOTAL profit=140.00 time_sum=852.93 stops_sum=7
Saved -> experiments\tables\greedy_k_hptoptw-j11a.csv
>> c:\Users\juann\anaconda3\python.exe c:\Users\juann\OneDrive - Pontificia Universidad Javeriana\Trabajo de Grado\ttdp-trabajo-grado\src\greedy_k_routes.py --file c:\Users\juann\OneDrive - Pontificia Universidad Javeriana\Trabajo de Grado\ttdp-trabajo-grado\data\synthetic\hptoptw-j11b.csv --name hptoptw-j11b --return-to-depot
[hptoptw-j11b] route=0 feasible=True profit=100.

,instance,k,route_id,feasible,profit,route_time,stops,compute_sec,route_nodes
0,hptoptw-j11a,2,0,True,100.0,607.540659,5,0.005088,"0,6,10,4,7,3,0"
1,hptoptw-j11a,2,1,True,40.0,245.393329,2,0.002256,"0,1,2,0"
2,hptoptw-j11b,2,0,True,100.0,479.079872,4,0.011230,"0,1,4,6,5,0"
3,hptoptw-j11b,2,1,True,50.0,483.294688,3,0.002276,"0,9,8,10,0"
4,hptoptw-j16a,3,0,True,110.0,725.401754,4,0.005452,"0,10,6,13,2,0"
